In [61]:
from PIL import Image
import os, glob, sys, numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

img_dir ="./test"
categories = ["Cat","Dog"]
np_classes = len(categories)

image_w = 64
image_h = 64


pixel = image_h * image_w * 3

X = []
y = []

# a = ['hong','gil','dong']
# b = list(enumerate(randomlist))
# c = dict(enumerate(randomlist))
# print(b) # 결과 # [(0, 'hong'), (1, 'gil'), (2, 'dong')]
# print(c) # 결과 # {0: 'hong', 1: 'gil', 2: 'dong'}

for idx, cat in enumerate(categories):
    img_dir_detail = img_dir + "/" + cat
    files = glob.glob(img_dir_detail+"/*.jpg")
    # glob() 함수는 인자로 받은 패턴과 이름이 일치하는 모든 파일과 디렉터리의 리스트를 반환합니다. 
    
    for i, f in enumerate(files):
        try:
            img = Image.open(f)
            img = img.convert("RGB")
            img = img.resize((image_w, image_h))
            data = np.asarray(img)
            #Y는 0 아니면 1이니까 idx값으로 넣는다.
            X.append(data)
            y.append(idx)
            if i % 300 == 0:
                print(cat, " : ", f)
        except:
            print(cat, str(i)+" 번째에서 에러 ")

X = np.array(X)
Y = np.array(y)


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)

xy = (X_train, X_test, Y_train, Y_test)
np.save("./numpy_data/binary_image_data.npy", xy)

Cat  :  ./test/Cat\0.jpg
Cat  :  ./test/Cat\1268.jpg
Cat  :  ./test/Cat\1548.jpg
Cat  :  ./test/Cat\378.jpg
Cat  :  ./test/Cat\648.jpg
Cat 1220 번째에서 에러 
Cat  :  ./test/Cat\918.jpg
Dog  :  ./test/Dog\0.jpg
Dog  :  ./test/Dog\1268.jpg
Dog  :  ./test/Dog\1547.jpg
Dog  :  ./test/Dog\377.jpg
Dog  :  ./test/Dog\647.jpg
Dog  :  ./test/Dog\917.jpg


In [62]:
import numpy as np
np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
np.load = np_load_old
print(np.__version__)

1.16.1


In [63]:
#import keras #taeyoon
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import keras.backend.tensorflow_backend as K

X_train, X_test, y_train, y_test = np.load("./numpy_data/binary_image_data.npy")
print(X_train.shape)
print(X_train.shape[0])
print(np.bincount(y_train))
print(np.bincount(y_test))

(2862, 64, 64, 3)
2862
[1420 1442]
[169 149]


In [64]:
image_w = 64
image_h = 64
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255


with K.tf_ops.device('/device:GPU:0'):
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding="same", input_shape=X_train.shape[1:], activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(32, (3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(64, (3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation="sigmoid"))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model_dir = './model'
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    model_path = model_dir + "/dog_cat_classify.model"
    
    checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=7)

In [65]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 8, 8, 64)         

In [66]:
history = model.fit(X_train, y_train, batch_size=64, epochs=10, validation_split=0.15, callbacks=[checkpoint, early_stopping])
#epochs=100는 너무 오래걸린다.

Epoch 1/10
38/38 [==============================] - ETA: 0s - loss: 0.6952 - accuracy: 0.5066
Epoch 00001: val_loss improved from inf to 0.68887, saving model to ./model\dog_cat_classify.model
INFO:tensorflow:Assets written to: ./model\dog_cat_classify.model\assets
38/38 [==============================] - 17s 460ms/step - loss: 0.6952 - accuracy: 0.5066 - val_loss: 0.6889 - val_accuracy: 0.6116
Epoch 2/10
38/38 [==============================] - ETA: 0s - loss: 0.6799 - accuracy: 0.5646
Epoch 00002: val_loss improved from 0.68887 to 0.65865, saving model to ./model\dog_cat_classify.model
INFO:tensorflow:Assets written to: ./model\dog_cat_classify.model\assets
38/38 [==============================] - 17s 441ms/step - loss: 0.6799 - accuracy: 0.5646 - val_loss: 0.6587 - val_accuracy: 0.6209
Epoch 3/10
38/38 [==============================] - ETA: 0s - loss: 0.6534 - accuracy: 0.6217
Epoch 00003: val_loss improved from 0.65865 to 0.64060, saving model to ./model\dog_cat_classify.model
INF

In [67]:
print("정확도 : %.2f " %(model.evaluate(X_test, y_test)[1]))

10/10 [==============================] - 0s 46ms/step - loss: 0.4583 - accuracy: 0.8113
정확도 : 0.81 


In [68]:
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['loss', 'val_loss', 'acc', 'val_acc'], loc='upper left')
# plt.show()

In [69]:
from PIL import Image
import os, glob, numpy as np
from keras.models import load_model

import tensorflow as tf

seed = 5
tf.random.set_seed(seed)
np.random.seed(seed)

caltech_dir = './Animal'


image_w = 64
image_h = 64

pixels = image_h * image_w * 3

X = []
filenames = []
files = glob.glob(caltech_dir+"/*/*.*")
for i, f in enumerate(files):
    img = Image.open(f)
    img = img.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)

    filenames.append(f)
    X.append(data)

X=np.array(X)
X = X.astype(float) / 255
#model = load_model('./model/dog_cat_classify.model')

prediction = model.predict(X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
cnt = 0
for i in prediction:
    if i >= 0.5: print("해당 " + filenames[cnt].split("\\")[1] + filenames[cnt].split("\\")[2] + "  이미지는 개 로 추정됩니다.")
    else : print("해당 " + filenames[cnt].split("\\")[1] + filenames[cnt].split("\\")[2] + "  이미지는 고양이 으로 추정됩니다.")
    cnt += 1

해당 CatCat1501.jpg  이미지는 개 로 추정됩니다.
해당 CatCat1502.jpg  이미지는 고양이 으로 추정됩니다.
해당 CatCat1503.jpg  이미지는 고양이 으로 추정됩니다.
해당 CatCat1504.jpg  이미지는 고양이 으로 추정됩니다.
해당 CatCat1505.jpg  이미지는 고양이 으로 추정됩니다.
해당 CatCat1506.jpg  이미지는 고양이 으로 추정됩니다.
해당 CatCat1507.jpg  이미지는 개 로 추정됩니다.
해당 CatCat1508.jpg  이미지는 고양이 으로 추정됩니다.
해당 CatCat1509.jpg  이미지는 고양이 으로 추정됩니다.
해당 CatCat1510.jpg  이미지는 고양이 으로 추정됩니다.
해당 DogDog1501.jpg  이미지는 개 로 추정됩니다.
해당 DogDog1502.jpg  이미지는 개 로 추정됩니다.
해당 DogDog1503.jpg  이미지는 개 로 추정됩니다.
해당 DogDog1504.jpg  이미지는 개 로 추정됩니다.
해당 DogDog1505.jpg  이미지는 고양이 으로 추정됩니다.
해당 DogDog1506.jpg  이미지는 고양이 으로 추정됩니다.
해당 DogDog1507.jpg  이미지는 개 로 추정됩니다.
해당 DogDog1508.jpg  이미지는 고양이 으로 추정됩니다.
해당 DogDog1509.jpg  이미지는 고양이 으로 추정됩니다.
해당 DogDog1510.jpg  이미지는 개 로 추정됩니다.


In [ ]:
0.51